In [ ]:
!pip install tensorflow

In [ ]:
import os
import cv2
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf


from tensorflow.keras.applications import ResNet50
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.preprocessing import image
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense

In [ ]:
# Ana veri klasörü
base_dir = 'drive/MyDrive/myData'
# Alt klasörler (sınıflar)
class_folders = ['buttons', 'dropdowns', 'textboxes', 'checkboxes', 'radioboxes']

In [ ]:
#Resimleri grayscale yapan kod parçası, ancak kullanılmadı.
for filename in os.listdir(base_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Resim formatlarını kontrol et
        img_path = os.path.join(base_dir, filename)  # Tam dosya yolunu oluştur
        img = Image.open(img_path).convert('L')  # Resmi aç ve grayscale'e çevir
        img.save(img_path)  # Aynı isimle kaydet


Tüm resimler grayscale yapıldı.


In [ ]:
os.makedirs('myData/train', exist_ok=True)
os.makedirs('myData/val', exist_ok=True)
os.makedirs('myData/test', exist_ok=True)

In [ ]:
for class_folder in class_folders:
    # Her sınıf için dizinler oluştur
    os.makedirs(f'myData/train/{class_folder}', exist_ok=True)
    os.makedirs(f'myData/val/{class_folder}', exist_ok=True)
    os.makedirs(f'myData/test/{class_folder}', exist_ok=True)

    # Sınıf klasöründeki tüm resimleri listele
    img_files = [f for f in os.listdir(os.path.join(base_dir, class_folder)) if os.path.isfile(os.path.join(base_dir, class_folder, f))]

    # Eğitim, doğrulama ve test setlerine ayır
    train_files, temp_files = train_test_split(img_files, test_size=0.30, random_state=42)
    val_files, test_files = train_test_split(temp_files, test_size=0.33, random_state=42)

    # Dosyaları uygun klasörlere kopyala
    for file_name in train_files:
        shutil.copy(os.path.join(base_dir, class_folder, file_name), os.path.join('myData/train', class_folder, file_name))
    for file_name in val_files:
        shutil.copy(os.path.join(base_dir, class_folder, file_name), os.path.join('myData/val', class_folder, file_name))
    for file_name in test_files:
        shutil.copy(os.path.join(base_dir, class_folder, file_name), os.path.join('myData/test', class_folder, file_name))

print("Veri seti başarıyla bölündü ve dosyalar kopyalandı.")

Veri seti başarıyla bölündü ve dosyalar kopyalandı.


In [ ]:
data_train_path = 'myData/train'
data_test_path = 'myData/test'
data_val_path = 'myData/val'

In [ ]:
img_width = 128
img_height = 256

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest',
)


data_train = train_datagen.flow_from_directory(
    data_train_path,
    shuffle = True,
    target_size = (img_width, img_height),
    batch_size = 32,
    class_mode = 'categorical',
    #color_mode='grayscale'
    )

Found 1855 images belonging to 5 classes.


In [ ]:
data_cat = data_train.class_indices
data_cat = {v: k for k, v in data_cat.items()}
data_cat

In [ ]:
val_test_datagen = ImageDataGenerator(
    rescale=1./255
)

data_val = val_test_datagen.flow_from_directory(
    data_val_path,
    shuffle = False,
    target_size = (img_width, img_height),
    batch_size = 32,
    class_mode = 'categorical',
    #color_mode='grayscale'
    )

In [ ]:
data_test = val_test_datagen.flow_from_directory(
    data_test_path,
    shuffle = False,
    target_size = (img_width, img_height),
    batch_size = 32,
    class_mode = 'categorical'
    )

In [ ]:
#Önceki denemeler bu model üzerinden yapıldı
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(img_width, img_height, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.4),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.4),

    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(5, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [ ]:
model = Sequential([
    base_model,               # Önceden eğitilmiş modelin eklenmesi
    Flatten(),                # Veriyi düzleştir
    Dense(512, activation='relu'),  # İlk dense katman (512 nöron)
    Dropout(0.5),             # %50 dropout
    Dense(256, activation='relu'),  # İkinci dense katman (256 nöron)
    Dropout(0.5),             # %50 dropout
    Dense(5, activation='softmax')  # Çıkış katmanı (5 sınıf)
])

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.0005),  # Adam optimizasyon algoritması
    loss='categorical_crossentropy',       # Kategorik çapraz entropi kaybı
    metrics=['accuracy']                   # Doğruluk metriği
)

In [ ]:
#Kullanılmadı
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3)

In [ ]:
history = model.fit(
    data_train,
    epochs=50,
    validation_data=data_val,
)

In [ ]:
test_loss, test_accuracy = model.evaluate(data_test)

print(f"Loss: {test_loss}")
print(f"Accuracy: {test_accuracy}")

In [ ]:
image = 'g.png'
image = tf.keras.utils.load_img(image, target_size=(img_width, img_height))
img_arr = tf.keras.utils.img_to_array(image)
img_bat = tf.expand_dims(img_arr, 0)

predict = model.predict(img_bat)

score = tf.nn.softmax(predict[0])

print('This belongs to {} class, accuracy: {:0.2f}'.format(data_cat[np.argmax(score)], 100 * np.max(score)))

In [ ]:
model.save('model.keras')